In [1]:
%matplotlib qt
%load_ext autoreload
%autoreload 2
# %reload_ext autoreload
from imports import *
repertory_memoire="/Users/margauxvandererven/Library/CloudStorage/OneDrive-UniversitéLibredeBruxelles/memoire/"

Dispersion is linear
Dispersion is linear
Dispersion is linear
Dispersion is linear


In [17]:
plt.figure(figsize=(20, 6))
# plt.plot(wavelen_visible, flux_visible, color='black', label='observé')

synthetic = syntspec("/Users/margauxvandererven/Unif/memoire_local/Turbospectrum_NLTE-master/COM/syntspec/synth_margaux/4000g1.0z-0.50m1.0t02a+0.20c+0.346n+0.00o+0.20r+0.00s+0.00.mod_4500-4700_vis.conv")
synthetic_norm = normalisation(wavelen_visible, flux_visible, 4600, 100)
# synthetic_norm2 = normalisation(synthetic_norm["z_wavelen"], synthetic_norm["flux_normalised"], 4600, 20)
plt.plot(synthetic_norm["z_wavelen"], synthetic_norm["flux_normalised"], color='black', label='observé')
# plt.plot(synthetic_norm["z_wavelen"], synthetic_norm["flux_normalised"], color='green', label='observé')
plt.plot(synthetic["wavelen"], synthetic["flux"], color='red', label='synthétique')
plt.hlines(1, 4500, 4700, color='lightgray', linestyle='--')
plt.xlim(4575, 4625)
plt.xlabel("Longueur d'onde (Å)")
plt.ylabel("Flux normalisé")
# plt.ylim(0.8, 200)
plt.legend()
plt.show()

In [18]:
# import numpy as np
# import scipy.interpolate as interp
# import matplotlib.pyplot as plt

def normalize_spectrum(wavelengths, flux, band_width=20):
    """
    Normalise le spectre en divisant par la médiane des 5 % plus grandes valeurs 
    dans une bande de 'band_width' Å autour de chaque point.
    """
    normalized_flux = np.zeros_like(flux)
    half_band = band_width / 2

    for i, wl in enumerate(wavelengths):
        mask = (wavelengths >= wl - half_band) & (wavelengths <= wl + half_band)
        if np.sum(mask) > 0:
            top_5_percent = np.percentile(flux[mask], 95)  # Seuil des 5% plus grands
            median_top_values = np.median(flux[mask][flux[mask] >= top_5_percent])  # Médiane des 5% plus grands
            normalized_flux[i] = flux[i] / median_top_values if median_top_values > 0 else flux[i]
        else:
            normalized_flux[i] = flux[i]

    return normalized_flux

def align_spectra(wl_obs, flux_obs, wl_syn, flux_syn, wl_ref, interp_kind="linear", align_band=20):
    """
    Aligne le spectre observé sur le spectre synthétique autour d'une longueur d'onde de référence `wl_ref`.
    1. Interpole le spectre observé sur la grille fine du synthétique.
    2. Normalise autour de `wl_ref` dans une bande de `align_band` Å.
    3. Ajuste verticalement le spectre observé pour l'aligner au synthétique.
    """
    wl_obs = np.array(wl_obs)
    flux_obs = np.array(flux_obs)
    wl_syn = np.array(wl_syn)
    flux_syn = np.array(flux_syn)
    obs_norm = normalisation(wl_obs,flux_obs, 4600, 100)

    interp_obs = interp.interp1d(obs_norm['z_wavelen'], obs_norm['flux_normalised'], kind=interp_kind, fill_value="extrapolate")
    flux_obs_interp_norm = interp_obs(wl_syn)

    half_band = align_band / 2
    mask = (wl_syn >= wl_ref - half_band) & (wl_syn <= wl_ref + half_band)

    if np.sum(mask) == 0:
        raise ValueError(f"Aucune donnée trouvée pour l'alignement dans [{wl_ref - half_band}, {wl_ref + half_band}] Å")

    diff_median = np.median(flux_obs_interp_norm[mask] - flux_syn[mask])  # Différence médiane
    aligned_flux = flux_obs_interp_norm - diff_median  # Correction

    return wl_syn, aligned_flux


synthetic = syntspec("/Users/margauxvandererven/Unif/memoire_local/Turbospectrum_NLTE-master/COM/syntspec/synth_margaux/4000g1.0z-0.50m1.0t02a+0.20c+0.346n+0.00o+0.20r+0.00s+0.00.mod_4500-4700_vis.conv")
synthetic_sansCH = syntspec("/Users/margauxvandererven/Unif/memoire_local/Turbospectrum_NLTE-master/COM/syntspec/synth_margaux/4000g1.0z-0.50m1.0t02a+0.20c+0.346n+0.00o+0.20r+0.00s+0.00.mod_4500-4700_vis_sans12CH.conv")
norm = normalisation(wavelen_visible, flux_visible, 4600, 100)

wl_obs = wavelen_visible
flux_obs = flux_visible
wl_syn = synthetic["wavelen"]  # Longueurs d'onde synthétiques
flux_syn = synthetic["flux"]  # Spectre synthétique

wl_syn_CH = synthetic_sansCH["wavelen"]  # Longueurs d'onde synthétiques
flux_syn_CH = synthetic_sansCH["flux"]  # Spectre synthétique
wl_syn_CH=np.array(wl_syn_CH)
flux_syn_CH=np.array(flux_syn_CH)
# Alignement du spectre observé sur la grille fine du synthétique
wl_aligned, flux_obs_aligned = align_spectra(wl_obs, flux_obs, wl_syn, flux_syn, wl_ref=4600)

# Affichage des résultats
plt.figure(figsize=(10, 5))
plt.hlines(1, 4500, 4700, color='gray', linestyle='--')

plt.plot(wl_syn, flux_obs_aligned, label="Observé normalisé et aligné", linewidth=1, color='black')

plt.plot(wl_syn, flux_syn, label="Synth avec 12CH", linewidth=1)
plt.plot(wl_syn_CH, flux_syn_CH, label="Synth sans 12CH", linewidth=1)
plt.plot(wl_syn, 1-np.abs(flux_syn-flux_syn_CH), label="Différence", linewidth=1)
# plt.plot(norm["z_wavelen"], norm["flux_normalised"], label="Observé normalisé", color='black', alpha=0.5)
plt.xlim(4500, 4700)
plt.ylim(0.0, 1.2)
# plt.scatter(wl_obs, flux_obs, label="Observé (original)", color="red", s=5, alpha=0.5)
plt.legend()
plt.xlabel("Longueur d'onde (Å)")
plt.ylabel("Flux")

plt.show()


In [ ]:
minimum = find_peaks_element("/Users/margauxvandererven/Library/CloudStorage/OneDrive-UniversitéLibredeBruxelles/memoire/BD-221742/synth_margaux/",
                   "4000g1.0z-0.50m1.0t02a+0.20c+0.346n+0.00o+0.20r+0.00s+0.00.mod_4000-6000_vis_Feabu_7.2.conv",
                   "4000g1.0z-0.50m1.0t02a+0.20c+0.346n+0.00o+0.20r+0.00s+0.00.mod_4000-6000_vis_sans12CH.conv")["minima"]
for i in minimum:
    if i < 5000:
        path_to_synth = "/Users/margauxvandererven/Library/CloudStorage/OneDrive-UniversitéLibredeBruxelles/memoire/BD-221742/synth_margaux/"
        synth={"4000g1.0z-0.50m1.0t02a+0.20c+0.346n+0.00o+0.20r+0.00s+0.00.mod_4000-6000_vis_Feabu_7.2.conv":"avec 12CH",
                    "4000g1.0z-0.50m1.0t02a+0.20c+0.346n+0.00o+0.20r+0.00s+0.00.mod_4000-6000_vis_sans12CH.conv": "sans 12CH"}
        zoom_lines({"":[i]}, path_to_synth, synth,stardata,2,lines_BD22_vis,gamme="visible")

/Users/margauxvandererven/Unif/memoire_local/scripts/zoom_raies.py:263: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  f = plt.figure(figsize=(10,y_size))


KeyboardInterrupt: 

In [ ]:
lines_BD22_vis = {"Fe I" : get_ew_atom(1e-4, 4000, gamme="visible", particular_element="Fe I")["wavelength"]}
# with open("../data_lines/lines_BD22_vis.json", "w") as f:
#     json.dump(lines_BD22_vis, f)

In [ ]:
get_ew_atom(1e-7, 4000, gamme="visible", particular_element="Fe II")["wavelength"]

[4122.668,
 4128.748,
 4173.461,
 4177.692,
 4178.862,
 4233.172,
 4258.154,
 4273.326,
 4296.572,
 4303.176,
 4314.31,
 4351.768,
 4385.387,
 4416.83,
 4489.183,
 4491.405,
 4508.288,
 4515.339,
 4520.224,
 4522.634,
 4534.168,
 4541.524,
 4549.474,
 4555.893,
 4576.34,
 4582.835,
 4583.837,
 4620.521,
 4629.339,
 4666.758,
 4731.453,
 4923.927,
 5018.44,
 5169.033,
 5197.577,
 5234.625,
 5276.002,
 5284.109,
 5316.615,
 5316.784,
 5362.869,
 5534.847,
 6516.08]